In [1]:
import pandas as pd
import glob
import time
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils
import keras

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sys
import numpy as np
sys.path.append('..')
from multivar_lstm.models import baseline_model
from multivar_lstm.lstm_data_generator import get_lstm_sub_data

In [3]:
from gensim.models import Word2Vec

In [4]:
def strtuple2arr(x):
    return np.array(x.replace('(', '').replace(')', '').split(','), dtype=np.float32)

In [5]:
modelpath = "./multivar_lstm/movie_w2v_final_model.model"

In [6]:
w2v_model = Word2Vec.load(modelpath)

In [7]:
def get_lstm_sub_data(df, modelpath, model):
    size = len(df)//1000 * 1000
    df = df.iloc[:size]
    w2v_model = model
    result_X = []
    Y = 0

    size = len(df)
    for col in df.columns:
        if 'MOVIE_ID' in col:
            #print(col)
            arr = w2v_model.wv[df[col].astype(int).astype(str).tolist()]
            result_X.append(arr)
            print(arr.shape)

        if 'scaled_duration' in col:
            #print(col)
            arr = df[col].values.reshape(size, 1)
            result_X.append(arr)
            print(arr.shape)

        if 'diff_day' in col:
            #print(col)
            arr = df[col].values.reshape(size, 1)
            result_X.append(arr)
            print(arr.shape)

        if 'week_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)
            print(arr.shape)

        if 'day_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)
            print(arr.shape)

        if 'month_coord' in col:
            #print(col)
            arr = np.array(df[col].apply(strtuple2arr).tolist())
            result_X.append(arr)
            print(arr.shape)

    X = np.concatenate(result_X, axis=1)
    print(X.shape)
    #X = X.reshape(X.shape[0], 10, 118)


    return X

In [8]:
dirpath = 'multivar_lstm/lstm_dataset/'
modelpath = 'multivar_lstm/movie_w2v_mincount_model.model'
file_list = glob.glob(dirpath+'/*')
model = baseline_model(shape=(1000, 10, 118), batch_size=1000)
epochs = int(10)
batch_size = 1000

rate = int(len(file_list)*1)

test_file_list = file_list[:rate]

In [12]:
stack_df = []

temp = pd.read_csv(test_file_list[0],index_col='Unnamed: 0')
stack_df.append(temp)
stack_df.append(temp.tail(800))

df = pd.concat(stack_df, axis=0)
df.info()


train_X = get_lstm_sub_data(df, modelpath, w2v_model)
train_X = train_X.reshape(99000,10,108)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99000 entries, 0 to 98199
Data columns (total 61 columns):
MOVIE_ID(t-0)           99000 non-null int64
scaled_duration(t-0)    99000 non-null float64
diff_day(t-0)           99000 non-null float64
week_coord(t-0)         99000 non-null object
day_coord(t-0)          99000 non-null object
month_coord(t-0)        99000 non-null object
MOVIE_ID(t-1)           99000 non-null float64
scaled_duration(t-1)    99000 non-null float64
diff_day(t-1)           99000 non-null float64
week_coord(t-1)         99000 non-null object
day_coord(t-1)          99000 non-null object
month_coord(t-1)        99000 non-null object
MOVIE_ID(t-2)           99000 non-null float64
scaled_duration(t-2)    99000 non-null float64
diff_day(t-2)           99000 non-null float64
week_coord(t-2)         99000 non-null object
day_coord(t-2)          99000 non-null object
month_coord(t-2)        99000 non-null object
MOVIE_ID(t-3)           99000 non-null float64
scaled_du

In [13]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('multivar_lstm/model/final_model.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('multivar_lstm/final_model.h5')

In [17]:
result = model.predict(train_X,batch_size=1000)[:98200]

In [22]:
n = 50
idx = (-result[0]).argsort()[:n]

In [26]:
def get_top_n(arr, n):
    return (-arr).argsort()[:n]

In [27]:
get_top_n(result[0],50)

array([3851, 4130, 1330, 5746, 5775, 5026, 4820, 4129, 2119, 2340, 6908,
       5382, 6411, 2608,  762, 4078, 7870, 3956,  583, 7908, 2051, 5694,
       7360, 4959, 3139,  753,  194, 3188,  824, 5015, 6428, 5726,  411,
       2929, 4009,  603, 7013, 4023, 8158, 1486, 2127, 5359, 4814, 7812,
        620,  704, 3092, 1400, 1092, 5162])

In [32]:
recm_arr = np.apply_along_axis(get_top_n,1,result, n=50)

In [33]:
recm_arr.shape

(98200, 50)

In [41]:
submit = pd.DataFrame(recm_arr)
submit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98200 entries, 0 to 98199
Data columns (total 50 columns):
0     98200 non-null int64
1     98200 non-null int64
2     98200 non-null int64
3     98200 non-null int64
4     98200 non-null int64
5     98200 non-null int64
6     98200 non-null int64
7     98200 non-null int64
8     98200 non-null int64
9     98200 non-null int64
10    98200 non-null int64
11    98200 non-null int64
12    98200 non-null int64
13    98200 non-null int64
14    98200 non-null int64
15    98200 non-null int64
16    98200 non-null int64
17    98200 non-null int64
18    98200 non-null int64
19    98200 non-null int64
20    98200 non-null int64
21    98200 non-null int64
22    98200 non-null int64
23    98200 non-null int64
24    98200 non-null int64
25    98200 non-null int64
26    98200 non-null int64
27    98200 non-null int64
28    98200 non-null int64
29    98200 non-null int64
30    98200 non-null int64
31    98200 non-null int64
32    98200 non-null int64


In [42]:
user_id = pd.DataFrame(temp['USER_ID'])
user_id.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98200 entries, 0 to 98199
Data columns (total 1 columns):
USER_ID    98200 non-null int64
dtypes: int64(1)
memory usage: 4.0 MB


In [43]:
submit = pd.concat([user_id, submit],axis=1)
submit.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98200 entries, 0 to 98199
Data columns (total 51 columns):
USER_ID    98200 non-null int64
0          98200 non-null int64
1          98200 non-null int64
2          98200 non-null int64
3          98200 non-null int64
4          98200 non-null int64
5          98200 non-null int64
6          98200 non-null int64
7          98200 non-null int64
8          98200 non-null int64
9          98200 non-null int64
10         98200 non-null int64
11         98200 non-null int64
12         98200 non-null int64
13         98200 non-null int64
14         98200 non-null int64
15         98200 non-null int64
16         98200 non-null int64
17         98200 non-null int64
18         98200 non-null int64
19         98200 non-null int64
20         98200 non-null int64
21         98200 non-null int64
22         98200 non-null int64
23         98200 non-null int64
24         98200 non-null int64
25         98200 non-null int64
26         98200 non-null i

In [44]:
submit.sort_values(['USER_ID'],inplace=True)

In [46]:
submit.to_csv('lstm_model_submit.csv',index=False,header=None)